# Проверка работы функций из src.utills и src.recommender

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from implicit.nearest_neighbours import ItemItemRecommender 

In [2]:
# импорт модулей src.utils и src.recommender

from src.utils import precision_at_k, prefilter_items, postfilter_items, popular_items, gen_dicts
from src.recommender import get_similar_items_recommendation, get_similar_users_recommendation

###  Проверка работы get_similar_items_recommendation
Берем топ-5 популярных покупок юзера --> находим по 1 наиболее близкому товару для них --> рекомендуем

In [3]:
# читаем данные из файла
data = pd.read_csv('transaction_data.csv')

# приводим названия столбцов к нижнему регистру и переименовываем два столбца
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3
# делим выборку на train и test
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# в тестовых данных для каждого пользователя находим список купленных товаров
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

# в обучающей выборке берем только 5000 самых популярных товаров
data_train = popular_items(data=data_train)

In [4]:
# строим матрицу c_ui
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
c_ui = user_item_matrix 
user_item_matrix = bm25_weight(user_item_matrix.T).T  # применям к c_ui матрице bm25 веса 

In [5]:
%%time

# обучаем ALS модель на матрице c_ui
model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                              show_progress=True)


Wall time: 992 ms


In [6]:
# делаем предсказание с помощью построенной модели для каждого пользователя
res = get_similar_items_recommendation(model, data_train, c_ui, N=5)

In [7]:
res.head(6)

,user_id,similar_items_rec
0,1,"[1028473, 927783, 1132231, 873916, 9526410]"
1,2,"[836793, 960318, 1133018, 880888, 819978]"
2,3,"[1075979, 960318, 1000736, 1076875, 886481]"
3,4,"[9526410, 1084310, 997128, 1105301, 1055155]"
4,5,"[1070820, 1010029, 833025, 12262778, 1105301]"
5,6,"[948650, 5569845, 889731, 890739, 1037863]"


###  Проверка работы get_similar_users_recommendation

Пользователь --> похожие пользователи --> рекомендовать те товары, которые купили похожие пользователи

In [8]:
userids = c_ui.index.values
itemids = c_ui.columns.values

id_to_userid, userid_to_id = gen_dicts(userids)
id_to_itemid, itemid_to_id = gen_dicts(itemids)

In [9]:
%%time
# обучаем item-item модель 
own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

own.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)


Wall time: 434 ms


In [10]:
%%time
# делаем предсказание с помощью построенной модели для каждого пользователя
# ждем исполнения минут 30-35

res1 = get_similar_users_recommendation(user_model=model, item_model=own, data=data_train, c_ui=c_ui)

Wall time: 35min 35s


In [11]:
res1.head(6)

,user_id,similar_users_rec
0,1,"[6533889, 1095275, 1138189, 1037840, 879504]"
1,2,"[6533889, 1114653, 885290, 915349, 1056509]"
2,3,"[1104224, 6533889, 1070820, 13511722, 1070803]"
3,4,"[6533889, 5570882, 889649, 959985, 1138292]"
4,5,"[6533889, 1039781, 1056651, 1070803, 1404121]"
5,6,"[6533889, 5585635, 849098, 1076875, 7167218]"


###  Проверка работы prefilter_items и postfilter_items

In [12]:
prefilter_items(data_train).head(4)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.0,1631,1,0.0,0.0


In [15]:
postfilter_items(data_train, res)